# <span style="color:#1f77b4">**Data Analytics 02 - DataFrame Schema**</span>

This notebook shapes raw CSV data into clean, typed DataFrames and summary views.



### Unity Catalog storage setup

Set the catalog/schema/volume for this project. Use an existing catalog, or create one in the UI if your metastore requires a managed location.


In [0]:
# Unity Catalog config for this project
dbutils.widgets.removeAll()
dbutils.widgets.text("CATALOG", "")
dbutils.widgets.text("SCHEMA", "default")
dbutils.widgets.text("VOLUME", "spark_lab")

catalog_widget = dbutils.widgets.get("CATALOG")
if catalog_widget:
    CATALOG = catalog_widget
else:
    # Prefer current catalog, otherwise pick the first non-system catalog
    current = spark.sql("SELECT current_catalog()").first()[0]
    catalogs = [r.catalog for r in spark.sql("SHOW CATALOGS").collect()]
    CATALOG = current if current not in ("system",) else next(c for c in catalogs if c not in ("system",))

SCHEMA = dbutils.widgets.get("SCHEMA")
VOLUME = dbutils.widgets.get("VOLUME")
BASE = f"dbfs:/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}"


In [0]:
# Ensure schema and volume exist
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME}")


DataFrame[]

### <span style="color:#1f77b4">**Loading CSV files into the Unity Catalog volume**</span>

Sync the raw CSV files into the Unity Catalog volume for this project.

- `dbutils.fs.mkdirs` creates the target directory.
- `dbutils.fs.cp` downloads the CSV files into the volume.


In [0]:
# Sync raw data files into the UC volume
sales_dir = f"{BASE}/sales"
dbutils.fs.rm(sales_dir, recurse=True)
dbutils.fs.mkdirs(sales_dir)
dbutils.fs.cp('https://raw.githubusercontent.com/Ch3rry-Pi3-Azure/DataBricks-Data-Analytics/refs/heads/main/data/2019_edited.csv', f'{BASE}/sales/2019_edited.csv')
dbutils.fs.cp('https://raw.githubusercontent.com/Ch3rry-Pi3-Azure/DataBricks-Data-Analytics/refs/heads/main/data/2020_edited.csv', f'{BASE}/sales/2020_edited.csv')
dbutils.fs.cp('https://raw.githubusercontent.com/Ch3rry-Pi3-Azure/DataBricks-Data-Analytics/refs/heads/main/data/2021_edited.csv', f'{BASE}/sales/2021_edited.csv')


True

### <span style="color:#1f77b4">**Loading the CSV files into a dataframe**</span>

Read the CSVs into a Spark DataFrame and preview the rows.

- `spark.read.load` reads files into a DataFrame.
- `display` shows a sample in Databricks.



In [0]:
# Load all CSVs into a Spark DataFrame
df = spark.read.load(BASE + "/sales/*.csv", format='csv')
display(df.limit(100))

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8
SO49171,1,2021-01-01,Mariah Foster,mariah21@adventure-works.com,"Road-250 Black, 48",1,2181.5625,174.525
SO49172,1,2021-01-01,Brian Howard,brian23@adventure-works.com,"Road-250 Red, 44",1,2443.35,195.468
SO49173,1,2021-01-01,Linda Alvarez,linda19@adventure-works.com,"Mountain-200 Silver, 38",1,2071.4196,165.7136
SO49174,1,2021-01-01,Gina Hernandez,gina4@adventure-works.com,"Mountain-200 Silver, 42",1,2071.4196,165.7136
SO49178,1,2021-01-01,Beth Ruiz,beth4@adventure-works.com,"Road-550-W Yellow, 44",1,1000.4375,80.035
SO49179,1,2021-01-01,Evan Ward,evan13@adventure-works.com,"Road-550-W Yellow, 38",1,1000.4375,80.035
SO49175,1,2021-01-01,Margaret Guo,margaret24@adventure-works.com,"Road-250 Red, 52",1,2443.35,195.468
SO49180,1,2021-01-01,Mitchell Yuan,mitchell6@adventure-works.com,"Road-650 Black, 58",1,782.99,62.6392
SO49176,1,2021-01-01,Shawn Sharma,shawn11@adventure-works.com,"Mountain-200 Silver, 46",1,2071.4196,165.7136
SO49177,1,2021-01-01,Barbara Chande,barbara44@adventure-works.com,"Mountain-200 Silver, 42",1,2071.4196,165.7136


### <span style="color:#1f77b4">**Defining Schema for the dataframe**</span>

Apply an explicit schema so dates, numbers, and strings parse consistently.

- `StructType` defines the full schema structure.
- `StructField` defines each column name, type, and nullability.



In [0]:
# Define an explicit schema for consistent types
from pyspark.sql.types import *
from pyspark.sql.functions import *
orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
])
df = spark.read.load(BASE + "/sales/*.csv", format='csv', schema=orderSchema)
display(df.limit(100))

SalesOrderNumber,SalesOrderLineNumber,OrderDate,CustomerName,Email,Item,Quantity,UnitPrice,Tax
SO49171,1,2021-01-01,Mariah Foster,mariah21@adventure-works.com,"Road-250 Black, 48",1,2181.5625,174.525
SO49172,1,2021-01-01,Brian Howard,brian23@adventure-works.com,"Road-250 Red, 44",1,2443.35,195.468
SO49173,1,2021-01-01,Linda Alvarez,linda19@adventure-works.com,"Mountain-200 Silver, 38",1,2071.4197,165.7136
SO49174,1,2021-01-01,Gina Hernandez,gina4@adventure-works.com,"Mountain-200 Silver, 42",1,2071.4197,165.7136
SO49178,1,2021-01-01,Beth Ruiz,beth4@adventure-works.com,"Road-550-W Yellow, 44",1,1000.4375,80.035
SO49179,1,2021-01-01,Evan Ward,evan13@adventure-works.com,"Road-550-W Yellow, 38",1,1000.4375,80.035
SO49175,1,2021-01-01,Margaret Guo,margaret24@adventure-works.com,"Road-250 Red, 52",1,2443.35,195.468
SO49180,1,2021-01-01,Mitchell Yuan,mitchell6@adventure-works.com,"Road-650 Black, 58",1,782.99,62.6392
SO49176,1,2021-01-01,Shawn Sharma,shawn11@adventure-works.com,"Mountain-200 Silver, 46",1,2071.4197,165.7136
SO49177,1,2021-01-01,Barbara Chande,barbara44@adventure-works.com,"Mountain-200 Silver, 42",1,2071.4197,165.7136


### <span style="color:#1f77b4">**Cleaning the Data**</span>

Remove duplicates and recompute tax for consistency.

- `dropDuplicates` removes repeated rows.
- `withColumn` and `col` derive and cast columns.



In [0]:
from pyspark.sql.functions import col
df = df.dropDuplicates()
df = df.withColumn('Tax', col('UnitPrice') * 0.08)
df = df.withColumn('Tax', col('Tax').cast("float"))

### <span style="color:#1f77b4">**Creating a new Dataframe**</span>

Build a customer-focused DataFrame with parsed names.

- `select` picks relevant columns.
- `split(...).getItem(...)` extracts first and last names.



In [0]:
customers_df = df.select("CustomerName", "Email", "Item", "Quantity")
customers_df = customers_df.withColumn("FirstName", split(customers_df["CustomerName"], " ").getItem(0))
customers_df = customers_df.withColumn("LastName", split(customers_df["CustomerName"], " ").getItem(1))

display(customers_df)

CustomerName,Email,Item,Quantity,FirstName,LastName
Leonard Nath,leonard20@adventure-works.com,"Mountain-200 Black, 42",1,Leonard,Nath
Terry Deng,terry4@adventure-works.com,"Road-250 Red, 44",1,Terry,Deng
Bonnie Chande,bonnie21@adventure-works.com,"Road-250 Red, 44",1,Bonnie,Chande
Garrett Sanchez,garrett22@adventure-works.com,"Road-650 Black, 62",1,Garrett,Sanchez
Shaun Shan,shaun11@adventure-works.com,"Road-250 Red, 52",1,Shaun,Shan
Bob Alan,bob6@adventure-works.com,"Road-650 Black, 58",1,Bob,Alan
Allison King,allison42@adventure-works.com,"Road-250 Red, 58",1,Allison,King
Alexa Travers,alexa11@adventure-works.com,"Road-250 Black, 58",1,Alexa,Travers
Barbara Kumar,barbara39@adventure-works.com,"Road-650 Black, 48",1,Barbara,Kumar
Seth Phillips,seth42@adventure-works.com,Fender Set - Mountain,1,Seth,Phillips


### <span style="color:#1f77b4">**Counting distinct customer entries**</span>

Compare total rows to distinct rows to assess duplicates.

- `count` returns row counts.
- `distinct` removes duplicates before counting.



In [0]:
print(customers_df.count())
print(customers_df.distinct().count())

32718
32200


### <span style="color:#1f77b4">**Creating a Product Sales Dataframe**</span>

Aggregate quantity sold by product.

- `groupBy` defines the grouping key.
- `sum` computes totals per product.



In [0]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()
display(productSales)

Item,sum(Quantity)
"Mountain-200 Black, 42",388
"Touring-1000 Yellow, 46",74
"Touring-1000 Blue, 54",67
"Short-Sleeve Classic Jersey, S",216
"Women's Mountain Shorts, S",146
"Long-Sleeve Logo Jersey, L",234
"Mountain-400-W Silver, 42",59
"Racing Socks, M",165
"Mountain-100 Silver, 42",42
"Mountain-200 Silver, 42",359


### <span style="color:#1f77b4">**Aggregating Yealy Sales**</span>

Summarize sales counts by year.

- `year` extracts the year from dates.
- `groupBy` and `orderBy` build a yearly summary.



In [0]:
yearlySales = df.select(year("OrderDate").alias("Year")).groupBy("Year").count().orderBy("Year")
display(yearlySales)

Year,count
2019,1201
2020,2733
2021,28784
